In [2]:
import numpy as np
from fn_linear import fn_linear
from fn_flatten import fn_flatten
from fn_relu import fn_relu
from fn_pool import fn_pool
from fn_conv import fn_conv
from fn_softmax import fn_softmax
from loss_crossentropy import loss_crossentropy 
from loss_euclidean import loss_euclidean

In [2]:
from calc_gradient import calc_gradient
from inference import inference
from init_layers import init_layers
from train import train
from pyc_update_weight import update_weights 
# ate_weights_SGD_momenum

In [3]:
import sys
sys.path += ['layers']
# from loss_euclidean import loss_euclidean

from load_MNIST_images import load_MNIST_images
from load_MNIST_labels import load_MNIST_labels

# Load training data
train_data = load_MNIST_images('train-images.idx3-ubyte')
print (train_data.shape)
train_label = load_MNIST_labels('train-labels.idx1-ubyte')
# Load testing data
test_data = load_MNIST_images('t10k-images.idx3-ubyte')
test_label = load_MNIST_labels('t10k-labels.idx1-ubyte')

(28, 28, 1, 60000)


In [4]:
from init_model import init_model

In [16]:
trainshape = train_data.shape
num_in = trainshape[0]*trainshape[1]*trainshape[2]*trainshape[3]

l = [init_layers('conv', {'filter_size': 2,
                          'filter_depth': 1,
                          'num_filters': 6}),
     init_layers('pool', {'filter_size': 2,
                          'stride': 2}),
     init_layers('conv', {'filter_size': 2,
                          'filter_depth': 6,
                          'num_filters': 16}),
     init_layers('pool', {'filter_size': 2,
                          'stride': 2}),
     init_layers('relu', {}),
     init_layers('flatten', {}),
     init_layers('linear', {'num_in': 576,
                    'num_out': 120}),
    init_layers('linear', {'num_in': 120,
                    'num_out': 84}),
     init_layers('linear', {'num_in': 84,
                            'num_out': 10}),
     init_layers('softmax', {})]

params = {
#     "learning_rate": .01,
#     "weight_decay": .0005,
    "learning_rate": .01,
    "weight_decay": .0005,
    "batch_size": 128,
    "save_file": 'model.npz'
}

model = init_model(l, list(train_data[:,:,:,0].shape), 10, True)


Input size:
[28, 28, 1]
Layer 0 output size: 
(26, 26, 6)
Layer 1 output size: 
(13, 13, 6)
Layer 2 output size: 
(12, 12, 16)
Layer 3 output size: 
(6, 6, 16)
Layer 4 output size: 
(6, 6, 16)
Layer 5 output size: 
(576,)
Layer 6 output size: 
(120,)
Layer 7 output size: 
(84,)
Layer 8 output size: 
(10,)
Final output size:
(10,)
Provided output size (should match above):
10
(Batch dimension not included)


In [17]:
input = train_data
max_model = model
numIters = 120
allmodels = []

label = train_label 
accuracys = np.zeros((numIters,))

lr_initial = params.get("learning_rate", .05)
# Weight decay
wd = params.get("weight_decay", .001)
# Batch size
batch_size = params.get("batch_size", 128)
# There is a good chance you will want to save your network model during/after
# training. It is up to you where you save and how often you choose to back up
# your model. By default the code saves the model in 'model.npz'.
save_file = params.get("save_file", 'model.npz')

# update_params will be passed to your update_weights function.
# This allows flexibility in case you want to implement extra features like momentum.
update_params = {"learning_rate": lr_initial,
                 "weight_decay": wd }

num_inputs = input.shape[-1]
loss = np.zeros((numIters,))

for i in range(numIters):
    #lr = lr_initial * np.exp(-np.log(5)/120*i)
#     update_params = {"learning_rate": lr,
#                  "weight_decay": wd }
    
    # if (i > 60) weight_decay = 0.005
    # TODO: One training iteration
    # Steps:
    #   (1) Select a subset of the input to use as a batch
    #   (2) Run inference on the batch
    #   (3) Calculate loss and determine accuracy
    #   (4) Calculate gradients
    #   (5) Update the weights of the model
    # Optionally,
    #   (1) Monitor the progress of training
    #   (2) Save your learnt model, using ``np.savez(save_file, **model)``
    batch_num = np.random.randint(num_inputs, size=batch_size)
    sample_input = input[:,:,:,batch_num]
    print(sample_input.shape)

    sample_label = label[batch_num]
    sample_output, sample_activations = inference(model, sample_input)
    loss[i], dv_gradient = loss_crossentropy(sample_output, sample_label, {}, True)

    print("Epoch: ", i, "loss: ", loss[i])
    count = 0.0
    for j in range (batch_size):
        if (np.argmax(sample_output[:,j]) == sample_label[j]):
            count +=1.0
    accuracys[i] = count/batch_size
    print("The accuracy is: ", accuracys[i])
    allmodels.append(model)
    
    sample_grads = calc_gradient(max_model, sample_input, sample_activations, dv_gradient)
    model = update_weights (max_model, sample_grads, update_params)
    
maxacc = np.argmax(accuracys)    
print("The max accuracy is: ", accuracys[maxacc])
np.savez("max_accuracy_model", **allmodels[maxacc]) 

minloss = np.argmin(loss)    
print("The min loss is: ", loss[minloss])
np.savez("min_loss_model", **allmodels[minloss]) 

(28, 28, 1, 128)
Epoch:  0 loss:  2.363266661363439
The accuracy is:  0.0546875
pyc update weight
(28, 28, 1, 128)
Epoch:  1 loss:  2.3117339132216195
The accuracy is:  0.078125
pyc update weight
(28, 28, 1, 128)
Epoch:  2 loss:  2.3086530286189078
The accuracy is:  0.109375
pyc update weight
(28, 28, 1, 128)
Epoch:  3 loss:  2.2921410759896923
The accuracy is:  0.0625
pyc update weight
(28, 28, 1, 128)
Epoch:  4 loss:  2.3354058552403423
The accuracy is:  0.1171875
pyc update weight
(28, 28, 1, 128)
Epoch:  5 loss:  2.3191271914846463
The accuracy is:  0.09375
pyc update weight
(28, 28, 1, 128)
Epoch:  6 loss:  2.3559596580817113
The accuracy is:  0.0625
pyc update weight
(28, 28, 1, 128)
Epoch:  7 loss:  2.3335407702092272
The accuracy is:  0.0703125
pyc update weight
(28, 28, 1, 128)
Epoch:  8 loss:  2.3131230043783595
The accuracy is:  0.078125
pyc update weight
(28, 28, 1, 128)
Epoch:  9 loss:  2.3367722266346647
The accuracy is:  0.09375
pyc update weight
(28, 28, 1, 128)
Epoch: 

SystemError: PyEval_EvalFrameEx returned a result with an error set

In [ ]:
input = train_data
model = np.load('max_accuracy_model.npz', allow_pickle=True)
model = dict(model) 
numIters = 120
allmodels2 = []

label = train_label 
accuracys2 = np.zeros((numIters,))

lr_initial = params.get("learning_rate", .05)
# Weight decay
wd = params.get("weight_decay", .001)
# Batch size
batch_size = params.get("batch_size", 128)
# There is a good chance you will want to save your network model during/after
# training. It is up to you where you save and how often you choose to back up
# your model. By default the code saves the model in 'model.npz'.
save_file = params.get("save_file", 'model.npz')
num_inputs = input.shape[-1]

loss2 = np.zeros((numIters,))
wd /= 1.6

for i in range(numIters):
    lr = lr_initial * np.exp(-np.log(5)/120*(i+90))
    update_params = {"learning_rate": lr,
                 "weight_decay": wd }
    # if (i > 60) weight_decay = 0.005
    # TODO: One training iteration
    # Steps:
    #   (1) Select a subset of the input to use as a batch
    #   (2) Run inference on the batch
    #   (3) Calculate loss and determine accuracy
    #   (4) Calculate gradients
    #   (5) Update the weights of the model
    # Optionally,
    #   (1) Monitor the progress of training
    #   (2) Save your learnt model, using ``np.savez(save_file, **model)``
    batch_num = np.random.randint(num_inputs, size=batch_size)
    sample_input = input[:,:,:,batch_num]
    print(sample_input.shape)

    sample_label = label[batch_num]
    sample_output, sample_activations = inference(model, sample_input)
    loss2[i], dv_gradient = loss_crossentropy(sample_output, sample_label, {}, True)

    print("Epoch: ", i, "loss: ", loss2[i])
    count = 0.0
    for j in range (batch_size):
        if (np.argmax(sample_output[:,j]) == sample_label[j]):
            count +=1.0
    accuracys2[i] = count/batch_size
    print("The accuracy is: ", accuracys2[i])
    allmodels2.append(model)
    
    sample_grads = calc_gradient(model, sample_input, sample_activations, dv_gradient)
    model = update_weights (model, sample_grads, update_params)
    
maxacc2 = np.argmax(accuracys2)    
print("The max accuracy is: ", accuracys2[maxacc2])
np.savez("max_accuracy_model_2", **allmodels2[maxacc2]) 

minloss2 = np.argmin(loss2)    
print("The min loss is: ", loss2[minloss2])
np.savez("min_loss_model_2", **allmodels2[minloss2])

In [ ]:
input = train_data
model = np.load('max_accuracy_model_2.npz', allow_pickle=True)
model = dict(model) 
numIters = 40
allmodels3 = []

label = train_label 
accuracys3 = np.zeros((numIters,))

lr_initial = 0.0005
# Weight decay
wd = 0.00005
# Batch size
batch_size = params.get("batch_size", 128)
# There is a good chance you will want to save your network model during/after
# training. It is up to you where you save and how often you choose to back up
# your model. By default the code saves the model in 'model.npz'.
save_file = params.get("save_file", 'model.npz')
num_inputs = input.shape[-1]

loss3 = np.zeros((numIters,))

for i in range(numIters):
    lr = lr_initial 
#     * np.exp(-np.log(5)/120*(i+90))
    update_params = {"learning_rate": lr,
                 "weight_decay": wd }
    # if (i > 60) weight_decay = 0.005
    # TODO: One training iteration
    # Steps:
    #   (1) Select a subset of the input to use as a batch
    #   (2) Run inference on the batch
    #   (3) Calculate loss and determine accuracy
    #   (4) Calculate gradients
    #   (5) Update the weights of the model
    # Optionally,
    #   (1) Monitor the progress of training
    #   (2) Save your learnt model, using ``np.savez(save_file, **model)``
    batch_num = np.random.randint(num_inputs, size=batch_size)
    sample_input = input[:,:,:,batch_num]
    print(sample_input.shape)

    sample_label = label[batch_num]
    sample_output, sample_activations = inference(model, sample_input)
    loss3[i], dv_gradient = loss_crossentropy(sample_output, sample_label, {}, True)

    print("Epoch: ", i, "loss: ", loss3[i])
    count = 0.0
    for j in range (batch_size):
        if (np.argmax(sample_output[:,j]) == sample_label[j]):
            count +=1.0
    accuracys3[i] = count/batch_size
    print("The accuracy is: ", accuracys3[i])
    allmodels3.append(model)
    if (loss3[i] > 0.39):
        sample_grads = calc_gradient(model, sample_input, sample_activations, dv_gradient)
        model = update_weights (model, sample_grads, update_params)
    
maxacc3 = np.argmax(accuracys3)    
print("The max accuracy is: ", accuracys3[maxacc3])
np.savez("max_accuracy_model_3", **allmodels3[maxacc3]) 

minloss3 = np.argmin(loss3)    
print("The min loss is: ", loss3[minloss3])
np.savez("min_loss_model_3", **allmodels3[minloss3])

In [ ]:
maxacc3 

In [13]:
model = np.load('min_loss_model_3.npz', allow_pickle=True)
model = dict(model)
test_output, activations = inference(model, test_data)
count = 0.0
for j in range (test_data.shape[-1]):
    if (np.argmax(test_output[:,j]) == test_label[j]):
        count +=1.0
print("The accuracy is: ", count/test_data.shape[-1])

The accuracy is:  0.9055


In [3]:
model = np.load('min_loss_model_3.npz', allow_pickle=True)

In [5]:
model = dict(model)

In [6]:
model

{'layers': array([{'fwd_fn': <function fn_conv at 0x113225158>, 'type': 'conv', 'params': {'W': array([[[[-0.23527176,  0.53225204, -0.39473971,  0.27195726,
           -0.19041166,  0.7254223 ]],
 
         [[-0.0884188 ,  0.59601797, -0.3627225 ,  0.24932023,
           -0.13738005,  0.68139493]]],
 
 
        [[[-0.16522687,  0.35622712, -0.43479344,  0.17045218,
           -0.27555627,  0.43658177]],
 
         [[-0.04005257,  0.47369281, -0.42534967,  0.06894007,
           -0.23382454,  0.46199382]]]]), 'b': array([[-0.00802005],
        [-0.1035436 ],
        [-0.06903803],
        [-0.06366293],
        [ 0.12886452],
        [ 0.03187634]])}, 'hyper_params': {'filter_size': 2, 'filter_depth': 1, 'num_filters': 6}},
        {'fwd_fn': <function fn_pool at 0x1132250d0>, 'type': 'pool', 'params': {'W': array([], dtype=float64), 'b': array([], dtype=float64)}, 'hyper_params': {'filter_size': 2, 'stride': 2}},
        {'fwd_fn': <function fn_conv at 0x113225158>, 'type': 'conv', 'p